# Processamento de texto

Iremos estudar brevemente como processar texto usando redes neurais. Como esse não é um objetivo do curso, utilizaremos a biblioteca Hugginface para fazer essa tarefa. Mas é importante entendermos o processamento realizado pela biblioteca.

### Pipeline do Hugginface

Primeiramente utilizaremos um pipeline do Hugginface, que é a interface de mais alto nível da biblioteca

Lista de modelos disponíveis para o pipeline: https://huggingface.co/models \
Lista de tarefas disponíveis para o pipeline: https://huggingface.co/docs/transformers/main_classes/pipelines#transformers.pipeline.task

In [1]:
from transformers import pipeline

pipe = pipeline(model='distilbert/distilbert-base-cased', task='feature-extraction', 
                return_tensors=True)

text = 'Some sentence to test the model'

In [2]:
y = pipe(text)
y

tensor([[[ 0.3386,  0.0341, -0.0293,  ..., -0.2339,  0.0973,  0.0785],
         [ 0.1333, -0.3066,  0.1266,  ...,  0.0262,  0.0969,  0.2786],
         [ 0.2536, -0.0148,  0.1565,  ..., -0.0567, -0.1064,  0.1481],
         ...,
         [-0.0250, -0.2859,  0.0105,  ...,  0.2711, -0.1073,  0.3281],
         [ 0.0498, -0.1095, -0.2095,  ...,  0.3505, -0.0599, -0.0365],
         [ 0.7945,  0.0952, -0.0333,  ..., -0.0862,  0.5560, -0.2026]]])

In [3]:
y[0].shape

torch.Size([8, 768])

A sentença é transformada em uma matriz de tamanho 8x768. Vamos entender como isso é feito.

O pipeline é composto por um tokenizador e uma rede neural:

In [4]:
tokenizer = pipe.tokenizer
model = pipe.model

print(tokenizer)
print(model)

DistilBertTokenizerFast(name_or_path='distilbert/distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
 

### Tokenizador

O tokenizador possui um dicionário que transforma trechos da sentença em um número:

In [5]:
vocab = tokenizer.vocab
# Imprime os 10 primeiros elementos do vocabulário
for idx, (key, value) in enumerate(vocab.items()):
    print(key, value)
    if idx==10:
        break


##low 6737
Chargers 21309
Edit 18269
potentially 9046
##sboro 27019
reelected 23575
##well 3192
tenth 8281
Aviv 12927
Reno 19139
regardless 8334


Algumas palavras do vocabulário são termos como `##ing`. Isso porque o modelo usado para tokenização (WordPiece) pode quebrar palavras em diferentes subpalavras que ocorrem com muita frequência. Por exemplo, a palavra `partying` é tokenizada como '['party', '##ing']', pois a palavra `party` ocorre com frequência de forma isolada e o final de palavra `ing` também é bem comum.

Isso possibilita que o vocabulário seja menor, pois não é necessário armazenar todas as palavras do idioma, apenas partes de palavras.

In [6]:
# Tamanho total do vocabulário
len(vocab)

28996

O processo de tokenização consiste em mapear cada palavra para o respectivo índice no vocabulário

In [7]:
[tokenizer.vocab[w] for w in text.split()]

[1789, 5650, 1106, 2774, 1103, 2235]

In [8]:
tokens = tokenizer.encode(text)
tokens

[101, 1789, 5650, 1106, 2774, 1103, 2235, 102]

São adicionados tokens especiais à sentença. No caso, o token de classificação e o token de separação entre sentenças:

In [9]:
tokenizer(text).tokens()

['[CLS]', 'Some', 'sentence', 'to', 'test', 'the', 'model', '[SEP]']

A forma padrão de usar o tokenizador é chamando a instância da classe:

In [10]:
texts = [text, 'Another sentence']
tokens = tokenizer(texts, return_tensors='pt', padding=True)
tokens

{'input_ids': tensor([[ 101, 1789, 5650, 1106, 2774, 1103, 2235,  102],
        [ 101, 2543, 5650,  102,    0,    0,    0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0]])}

`input_ids` são os tokens gerados para cada sentença. Sentenças curtas são preenchidas com 0 para ficarem com o mesmo tamanho que a maior sentença. 

O modelo BERT também possui a chamada `attention_mask`, que é uma máscara que indica quais tokens o modelo pode utilizar para fazer previsões. No nosso caso a máscara será usada somente para ignorar tokens de padding.

### Embedding

Os ids de tokens são transformados em vetores de valores. Isso é feito através de uma matriz de tamanho `len(vocab)x768`, na qual cada linha corresponde a um token.

In [12]:
embedding = model.embeddings(tokens['input_ids'])
embedding.shape

torch.Size([2, 8, 768])

A matriz de embedding inicialmente é aleatória. Ela é treinada juntamente com o modelo e otimizada para fornecer os melhores embeddings possíveis para a tarefa na qual o modelo é treinado. O processo é feito da seguinte forma:

In [13]:
import torch

embedding = torch.rand(len(vocab), 768, requires_grad=True)

# Embedding da sentença
x = embedding[tokenizer.encode(text)]
x.shape

# Uso da sentença para treinar o modelo 
#...

torch.Size([8, 768])

### Modelo BERT

Para aplicar o modelo nos tokens, basta fazermos:

In [14]:
res = model(**tokens)
res[0].shape

torch.Size([2, 8, 768])

O modelo gera duas matrizes, uma para cada sentença. Cada sentença possui 8 tokens. Para cada token, são gerados 768 atributos. No caso do modelo BERT, é comum utilizar os atributos gerados para o token de classificação (CLS) para caracterizar a sentença como um todo.

In [15]:
text_features = res[0][:,0]
text_features.shape

torch.Size([2, 768])

Utilização da GPU

In [16]:
model.to('cuda')
# A tokenização é feita na CPU
tokens = tokenizer(texts, return_tensors='pt', padding=True)
# Envia os tensores gerados para a GPU
tokens = tokens.to('cuda')
# Aplica o modelo
res = model(**tokens)

### Codificador de texto

Vamos criar um modelo para codificar textos. Esse modelo será utilizado junto com um modelo de imagens.

In [17]:
from torch import nn
from transformers import pipeline

class TextEncoder(nn.Module):

    def __init__(self):
        super().__init__()

        # Carrega o pipeline do Hugginface, que inclui um tokenizador e
        # um modelo de classificação de texto
        pipe = pipeline(model='distilbert/distilbert-base-cased', task='feature-extraction')
        tokenizer = pipe.tokenizer
        model = pipe.model

        self.tokenizer = tokenizer
        self.model = model
        # Índice do token associado à classificação
        self.cls_token_id = 0
        # Dimensão de saída do modelo distilbert
        self.feature_dim = 768

    def forward(self, text):

        # Se houver uma lista de textos, é preciso preencher com zeros
        # para deixá-los com mesmo tamanho
        padding = isinstance(text, (list, tuple))

        tokens = self.tokenizer(text, return_tensors='pt', padding=padding)

        # Envia o texto tokenizado para o mesmo device que o modelo
        tokens = tokens.to(self.model.device)
        res = self.model(**tokens)[0]

        # Acessa os atributos associados com o token de classificação
        features = res[:, self.cls_token_id]
        
        return features

text_encoder = TextEncoder()
features = text_encoder(texts)
features.shape

torch.Size([2, 768])